In [1]:
import cv2
import numpy as np
import os
import glob
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
import random
import splitfolders

In [2]:
faceImagesDir = os.path.join('./Dataset/face_images')
img_dir = os.path.join('./Dataset/face_images/train/face_images')
aug_dir = os.path.join('./Dataset/augment_images')
allTrain_dir = os.path.join('./Dataset/all_images')
labTrain_dir = os.path.join('./Dataset/lab_images')
lTrain_dir = os.path.join('./Dataset/l_images')
aTrain_dir = os.path.join('./Dataset/a_images')
bTrain_dir = os.path.join('./Dataset/b_images')
test_dir = os.path.join('./Dataset/face_images/test/face_images')
allTest_dir = os.path.join('./Dataset/all_images/test')
labTest_dir = os.path.join('./Dataset/lab_images/test')
lTest_dir = os.path.join('./Dataset/l_images/test')
aTest_dir = os.path.join('./Dataset/a_images/test')
bTest_dir = os.path.join('./Dataset/b_images/test')

In [3]:
splitfolders.ratio(faceImagesDir, # The location of dataset
                   output=faceImagesDir, # The output location
                   seed=42, # The number of seed
                   ratio=(.9, 0, .1), # The ratio of splited dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )

Data Augumentation

In [4]:
horizontalFlip = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=1),
])

verticalFlip = transforms.Compose([
    transforms.ToPILImage(),    
    transforms.RandomVerticalFlip(p=1),
])

twoFlips = transforms.Compose([
    transforms.ToPILImage(),   
    transforms.RandomVerticalFlip(p=1),
    transforms.RandomHorizontalFlip(p=1),
])

randomCrop = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop((64, 64)),    
])


def augment_data(img_dir,aug_dir):
    image_list = glob.glob(os.path.join(img_dir,"*.jpg"))    
    for i in image_list:
        image = plt.imread(i)
        randomValue = round(random.uniform(0.6, 1.0), 1)
        scaledImage = (image/255) + (randomValue, randomValue, randomValue); 
        
        image = Image.fromarray(image).convert('RGB')     
        image.save((i.replace(img_dir,aug_dir)).replace(".jpg","_0.jpg"))        
        image = np.asarray(image).astype(np.uint8) 
         
        horizontalFlip(image).save((i.replace(img_dir,aug_dir)).replace(".jpg","_1.jpg"))
        verticalFlip(image).save((i.replace(img_dir,aug_dir)).replace(".jpg","_2.jpg"))
        randomCrop(image).save((i.replace(img_dir,aug_dir)).replace(".jpg","_3.jpg"))
        twoFlips(image).save((i.replace(img_dir,aug_dir)).replace(".jpg","_4.jpg"))
        
        scaledImage = Image.fromarray((scaledImage * 255).astype(np.uint8)).convert('RGB')        
        scaledImage.save((i.replace(img_dir,aug_dir)).replace(".jpg","_5.jpg"))
        scaledImage = np.asarray(scaledImage)
        
        horizontalFlip(scaledImage).save((i.replace(img_dir,aug_dir)).replace(".jpg","_6.jpg"))
        verticalFlip(scaledImage).save((i.replace(img_dir,aug_dir)).replace(".jpg","_7.jpg"))
        randomCrop(scaledImage).save((i.replace(img_dir,aug_dir)).replace(".jpg","_8.jpg"))
        twoFlips(scaledImage).save((i.replace(img_dir,aug_dir)).replace(".jpg","_9.jpg"))

In [5]:
augment_data(img_dir, aug_dir)

Load Data using cv2.imread followed by BGR to LAB conversion and split L*a*b color image to L, a, b image folders respectively. 

In [6]:
def pre_process(inp_dir, l_dir, a_dir, b_dir, lab_dir, all_dir): 
    files = glob.glob(os.path.join(inp_dir,"*.jpg"))
    data = []
    labData = []
    for i in files:
        img = cv2.imread(str(i))
        img = cv2.resize(img, (128,128))
        imgAug = img[:, :, [2, 1, 0]]
        imageLAB = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    #     cv2.imshow('image', imgAug) 
    #     cv2.imshow('image1', imageLAB) 
        L,a,b=cv2.split(imageLAB) 
    #     cv2.imshow('L',L) 
    #     cv2.imshow('a',a) 
    #     cv2.imshow('b',b)
    #     cv2.waitKey()
        Image.fromarray(L).save((i.replace(inp_dir,l_dir)).replace(".jpg","_L.jpg"))
        Image.fromarray(a).save((i.replace(inp_dir,a_dir)).replace(".jpg","_a.jpg"))
        Image.fromarray(b).save((i.replace(inp_dir,b_dir)).replace(".jpg","_b.jpg"))   
        Image.fromarray(imageLAB).save((i.replace(inp_dir,lab_dir)).replace(".jpg","_lab.jpg"))
        Image.fromarray(imageLAB).save((i.replace(inp_dir,all_dir)).replace(".jpg","_lab.jpg")) 
        Image.fromarray(L).save((i.replace(inp_dir,all_dir)).replace(".jpg","_L.jpg"))
        Image.fromarray(a).save((i.replace(inp_dir,all_dir)).replace(".jpg","_a.jpg"))
        Image.fromarray(b).save((i.replace(inp_dir,all_dir)).replace(".jpg","_b.jpg"))
        Image.fromarray(imgAug).save((i.replace(inp_dir,all_dir)).replace(".jpg","_aug.jpg"))
        labData.append(imageLAB)
        data.append(imgAug)
    #     cv2.destroyAllWindows()
    return data, labData

In [7]:
#L*a*b color space conversion for training dataset
data, labData = pre_process(aug_dir, lTrain_dir, aTrain_dir, bTrain_dir, labTrain_dir, allTrain_dir)
#L*a*b color space conversion for testing dataset
pre_process(test_dir, lTest_dir, aTest_dir, bTest_dir, labTest_dir, allTest_dir)

([array([[[50, 34, 21],
          [54, 38, 25],
          [57, 39, 25],
          ...,
          [56, 41, 34],
          [57, 40, 33],
          [57, 40, 33]],
  
         [[52, 36, 23],
          [54, 38, 25],
          [55, 37, 25],
          ...,
          [52, 37, 30],
          [55, 40, 33],
          [58, 41, 34]],
  
         [[52, 34, 22],
          [54, 36, 24],
          [54, 38, 25],
          ...,
          [48, 37, 31],
          [54, 39, 34],
          [56, 41, 36]],
  
         ...,
  
         [[54, 49, 45],
          [52, 47, 43],
          [53, 45, 42],
          ...,
          [93, 66, 57],
          [91, 64, 55],
          [88, 61, 52]],
  
         [[56, 53, 48],
          [51, 48, 43],
          [48, 43, 37],
          ...,
          [92, 65, 56],
          [88, 61, 52],
          [89, 62, 53]],
  
         [[56, 53, 48],
          [52, 49, 44],
          [51, 46, 40],
          ...,
          [92, 65, 56],
          [90, 63, 54],
          [91, 64, 55]]], dtype=u

Defaulting the tensor datatype to 32-bit float.

In [8]:
dataArray=np.asarray(data)
dataArray=np.asarray(labData)
torch.set_default_tensor_type(torch.FloatTensor)

Transforms the input to tensor of dimention (number_of_images, channel, 128, 128) and shuffles the tensor values using torch.randperm

In [9]:
transform = transforms.Compose([transforms.ToTensor()])
train_tensors = []
for arr in dataArray:
    train_tensors.append(transform(arr))  

result = torch.stack(train_tensors)

#Shuffle
output = torch.randperm(result.size()[0])
result1 = result[output]
print(result1.size())

torch.Size([6740, 3, 128, 128])
